In [52]:
import pandas as pd
import numpy as np
df = pd.read_csv("./1029_attack.csv")
df

,Time,Source,Destination,Protocol,Length,ID,Data
0,0.000000,NaN,NaN,CAN,32,360 (0x168),8000080012504000
1,0.001220,NaN,NaN,CAN,32,1416 (0x588),00c6000040001000
2,0.002423,NaN,NaN,CAN,32,1417 (0x589),000000ff9c000000
3,0.003685,NaN,NaN,CAN,32,376 (0x178),0000000000000000
4,0.011932,NaN,NaN,CAN,32,257 (0x101),9000000000000100
...,...,...,...,...,...,...,...
163956,641.001177,NaN,NaN,CAN,32,327 (0x147),0000000000000000
163957,641.010075,NaN,NaN,CAN,32,274 (0x112),c000001f04800000
163958,641.012440,NaN,NaN,CAN,32,392 (0x188),98a0000000000000
163959,641.014382,NaN,NaN,CAN,32,321 (0x141),3000000000000000


In [54]:
# NaN값 0으로 채우기
df = df.fillna(0)
df

,Time,Source,Destination,Protocol,Length,ID,Data
0,0.000000,0.0,0.0,CAN,32,360 (0x168),8000080012504000
1,0.001220,0.0,0.0,CAN,32,1416 (0x588),00c6000040001000
2,0.002423,0.0,0.0,CAN,32,1417 (0x589),000000ff9c000000
3,0.003685,0.0,0.0,CAN,32,376 (0x178),0000000000000000
4,0.011932,0.0,0.0,CAN,32,257 (0x101),9000000000000100
...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,327 (0x147),0000000000000000
163957,641.010075,0.0,0.0,CAN,32,274 (0x112),c000001f04800000
163958,641.012440,0.0,0.0,CAN,32,392 (0x188),98a0000000000000
163959,641.014382,0.0,0.0,CAN,32,321 (0x141),3000000000000000


In [56]:
# ID를 360(0x168) => 168로 바꾸기

import re

# 데이터 로드
df = pd.read_csv('labeling진행중.csv')

# ID 열에서 '0x' 뒤의 세 자리 16진수 값만 추출
df['ID'] = df['ID'].apply(lambda x: re.search(r'0x([0-9a-fA-F]{3})', str(x)).group(1) if pd.notna(x) and re.search(r'0x([0-9a-fA-F]{3})', str(x)) else x)

# 결과 확인
print(df[['ID']].head())
df

    ID
0  168
1  588
2  589
3  178
4  101


,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0
...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0


## Feature 선정 

In [60]:
# 1번 feature : Can Data의 엔트로피 계산

import pandas as pd
from scipy.stats import entropy
from collections import Counter

# 슬라이딩 윈도우 방식으로 Data 열을 기준으로 엔트로피 계산
window_size = 20
step_size = 10
entropy_values = []

# 엔트로피 계산 함수
def calculate_entropy(window):
    # 문자열을 빈도수 카운트로 변환하고, 이를 확률 분포로 변환
    counts = Counter(window)
    probabilities = [count / sum(counts.values()) for count in counts.values()]
    return entropy(probabilities, base=2)  # base=2로 설정하여 비트를 기준으로 엔트로피 계산

for i in range(0, len(df) - window_size + 1, step_size):
    # Data 열에서 20개씩 묶음
    window = df['Data'].iloc[i:i + window_size]
    ent = calculate_entropy(window)
    print(f"Window {i // step_size + 1}: Entropy = {ent}")  # 각 윈도우별 엔트로피 값 출력
    entropy_values.extend([ent] * step_size)  # 엔트로피 값을 step_size 길이만큼 반복

# 남은 데이터가 있으면 마지막 엔트로피 값을 채워줌
if len(entropy_values) < len(df):
    entropy_values.extend([entropy_values[-1]] * (len(df) - len(entropy_values)))

# 엔트로피 값을 df에 추가
df['Entropy'] = entropy_values

# 결과 확인
print(df[['Data', 'Entropy']].head())

df


Window 1: Entropy = 3.3393538721672007
Window 2: Entropy = 3.5464393446710147
Window 3: Entropy = 3.9219280948873627
Window 4: Entropy = 3.641446071165521
Window 5: Entropy = 3.3393538721672007
Window 6: Entropy = 3.7414460711655213
Window 7: Entropy = 4.0841837197791895
Window 8: Entropy = 3.2393538721672006
Window 9: Entropy = 3.3393538721672007
Window 10: Entropy = 4.0841837197791895
Window 11: Entropy = 3.9219280948873627
Window 12: Entropy = 2.895461844238321
Window 13: Entropy = 3.5464393446710147
Window 14: Entropy = 4.221928094887361
Window 15: Entropy = 3.821928094887363
Window 16: Entropy = 3.121928094887362
Window 17: Entropy = 3.5464393446710147
Window 18: Entropy = 4.0841837197791895
Window 19: Entropy = 3.4464393446710146
Window 20: Entropy = 3.121928094887362
Window 21: Entropy = 3.5464393446710147
Window 22: Entropy = 3.9219280948873627
Window 23: Entropy = 3.4464393446710146
Window 24: Entropy = 3.3393538721672007
Window 25: Entropy = 3.7414460711655213
Window 26: Entr

,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354
...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446


In [61]:
df

,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354
...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446


In [62]:
# 2번 feature : 똑같은 데이터 들어오면 1로 출력

# 'Replay Attack' 열 초기화
df['Same Data'] = '0'

# 허용 오차 설정 (예: 0.1초)
time_threshold = 0.1

# Timestamp를 기준으로 데이터 정렬
df = df.sort_values(by='Time').reset_index(drop=True)

# Replay Attack 감지
for i in range(1, len(df)):
    # 이전 행과 현재 행을 비교
    if (df.loc[i, 'ID'] == df.loc[i - 1, 'ID'] and 
        df.loc[i, 'Data'] == df.loc[i - 1, 'Data'] and
        abs(df.loc[i, 'Time'] - df.loc[i - 1, 'Time']) <= time_threshold):
        
        # Replay Attack 라벨링
        df.loc[i, 'Same Data'] = '1'
        df.loc[i - 1, 'Same Data'] = '1'  # 이전 항목도 Replay Attack으로 표시
df

,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354
...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446


In [63]:
# 3번 feature : Data의 내용이 fffffffffffffffff

# 우리가 정한 attack data를 감지해서 있으면 1 아니면 0으로 판별한다
attack_datas = ['ffffffffffffffff']
df['Strange Data'] = np.where(df['Data'].isin(attack_datas), '1','0')
df


,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354
...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446


In [72]:
df.to_csv('1029_labeling.csv', index = False, encoding = 'utf-8')

In [65]:
df = pd.read_csv('labeling진행중.csv')
df

,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354
...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446


## csv 파일 만들기(fuzzing_messages.txt, replaying_messages.txt)

In [ ]:
## fuzzing label만들기 위한 csv파일 만들기

import pandas as pd
import re

# 결과를 저장할 빈 리스트 생성
data = []

# 정규 표현식을 사용해 필요한 값을 추출
pattern = r"Timestamp:\s+([\d.]+)\s+ID:\s+(\w+)\s+\w+\s+\w+\s+DL:\s+(\d+)\s+(.+)"

# 텍스트 파일 열기
with open("fuzzing_messages.txt", "r") as file:
    for line in file:
        # 정규 표현식으로 각 값 추출
        match = re.search(pattern, line)
        if match:
            timestamp_value = match.group(1)  # Timestamp 값
            id_value = match.group(2)         # ID 값
            dl_value = match.group(3)         # DL 값
            data_value = match.group(4)       # Data 값

            # 각 값을 딕셔너리로 저장하여 리스트에 추가
            data.append({
                "Timestamp": timestamp_value,
                "ID": id_value,
                "DL": dl_value,
                "Data": data_value
            })

# 리스트를 DataFrame으로 변환
df = pd.DataFrame(data)

# DataFrame을 CSV 파일로 저장
df.to_csv("fuzzing_messages.csv", index=False)


In [ ]:
import pandas as pd
import re

# 결과를 저장할 빈 리스트 생성
data = []

# 정규 표현식을 사용해 필요한 값을 추출
pattern = r"Timestamp:\s+([\d.]+)\s+ID:\s+(\w+)\s+\w+\s+\w+\s+DL:\s+(\d+)\s+(.+)"

# 텍스트 파일 열기
with open("replaying_messages.txt", "r") as file:
    for line in file:
        # 정규 표현식으로 각 값 추출
        match = re.search(pattern, line)
        if match:
            timestamp_value = match.group(1)  # Timestamp 값
            id_value = match.group(2)         # ID 값
            dl_value = match.group(3)         # DL 값
            data_value = match.group(4)       # Data 값

            # 'Channel: ...' 제거
            data_value = re.sub(r"\s+Channel:.*", "", data_value).strip()

            # 각 값을 딕셔너리로 저장하여 리스트에 추가
            data.append({
                "Timestamp": timestamp_value,
                "ID": id_value,
                "DL": dl_value,
                "Data": data_value
            })

# 리스트를 DataFrame으로 변환
df = pd.DataFrame(data)

# DataFrame을 CSV 파일로 저장
df.to_csv("replaying_messages.csv", index=False)


In [165]:
import pandas as pd
from datetime import datetime, timezone

# CSV 파일 읽기
df = pd.read_csv('replaying_messages.csv')

# Unix 타임스탬프를 사람이 읽을 수 있는 형식으로 변환
df['Readable_Time'] = df['Timestamp'].apply(lambda x: datetime.fromtimestamp(x, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))

# 결과 출력
print(df[['Timestamp', 'Readable_Time']])


        Timestamp        Readable_Time
0    1.730195e+09  2024-10-29 09:36:10
1    1.730195e+09  2024-10-29 09:36:10
2    1.730195e+09  2024-10-29 09:36:10
3    1.730195e+09  2024-10-29 09:36:10
4    1.730195e+09  2024-10-29 09:36:10
..            ...                  ...
589  1.730195e+09  2024-10-29 09:46:22
590  1.730195e+09  2024-10-29 09:46:22
591  1.730195e+09  2024-10-29 09:46:22
592  1.730195e+09  2024-10-29 09:46:22
593  1.730195e+09  2024-10-29 09:46:22

[594 rows x 2 columns]


In [167]:
df.to_csv("replaying_messages.csv", index=False)

## 정답 레이블 달아주기

In [145]:
# 데이터 로드
df = pd.read_csv('1029_labeling.csv')
df

,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354
...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446


In [147]:
df_fuzzing = pd.read_csv("./fuzzing_messages.csv")
df_fuzzing

,Timestamp,ID,DL,Data
0,0.0,fb5,8,f1 a1 13 02 80 fb 59 c5
1,0.0,0f8,8,c3 ec 7a 7a 4f fb e7 69
2,0.0,e47,8,18 d3 23 4c 96 e6 53 ee
3,0.0,039,8,c2 5a 0e 3f c8 e2 99 1e
4,0.0,19d,8,2b 39 ad 4d a7 f4 f7 83
...,...,...,...,...
6995,0.0,ddb,8,0e e5 c4 8a b0 aa 7d dc
6996,0.0,4e5,8,b2 6a 47 e9 5f ad 63 b4
6997,0.0,ece,8,2c ea d0 e1 ff 2b fd c4
6998,0.0,812,8,23 df 4e 40 7f 98 2a 45


In [149]:
df_replaying = pd.read_csv("./replaying_messages.csv")
df_replaying

,Timestamp,ID,DL,Data
0,1.730195e+09,188,8,98 a0 00 00 00 00 00 00
1,1.730195e+09,104,8,00 40 02 2c 00 00 00 00
2,1.730195e+09,112,8,d0 00 00 1f 04 80 00 00
3,1.730195e+09,189,8,21 00 00 04 00 00 00 00
4,1.730195e+09,19b,8,00 40 00 00 00 00 00 00
...,...,...,...,...
589,1.730195e+09,13c,8,00 00 00 00 00 00 50 00
590,1.730195e+09,19b,8,00 40 00 00 00 00 00 00
591,1.730195e+09,120,8,00 00 28 12 10 80 70 00
592,1.730195e+09,40a,8,12 02 00 00 00 00 ff ff


# Dos 공격 레이블 달기

In [152]:
# DoS 공격 레이블 달기 (DoS 조건을 먼저 적용)
df['Dos Attack'] = df['Data'].apply(lambda x: 'DoS' if x == 'ffffffffffffffff' else 'Normal')
print(df['Dos Attack'].value_counts())
df

Dos Attack
Normal    139961
DoS        24000
Name: count, dtype: int64


,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy,Dos Attack
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354,Normal
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354,Normal
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354,Normal
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354,Normal
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354,Normal
...,...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446,Normal
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446,Normal
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446,Normal
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446,Normal


# Fuzzing 레이블 달기

In [155]:
# 공백 제거 및 소문자로 변환하여 데이터 포맷 일치
df['Data'] = df['Data'].str.replace(" ", "").str.lower()
df_fuzzing['Data'] = df_fuzzing['Data'].str.replace(" ", "").str.lower()
df_fuzzing['ID'] = df_fuzzing['ID'].astype(str).str.lower()

# fuzzing_df의 ID와 Data 쌍을 set으로 만들어 빠르게 조회
fuzzing_set = set(zip(df_fuzzing['ID'], df_fuzzing['Data']))

# Fuzzing 공격 레이블 달기
df['Fuzzing Attack'] = df.apply(lambda row: 'Fuzzing' if (row['ID'], row['Data']) in fuzzing_set else 'Normal', axis=1)

# 일치하지 않는 항목 확인
unmatched_fuzzing = fuzzing_set - set(zip(df['ID'], df['Data']))
print(f"일치하지 않는 Fuzzing 항목 수: {len(unmatched_fuzzing)}")
print("일치하지 않는 Fuzzing 항목 샘플:", list(unmatched_fuzzing)[:10])

# 결과 확인
print(df['Fuzzing Attack'].value_counts())
df


일치하지 않는 Fuzzing 항목 수: 3514
일치하지 않는 Fuzzing 항목 샘플: [('9d0', '2e493a1e41e3af57'), ('fae', '2c97ad6f41b12140'), ('c63', '6883fbbb0b2da0f4'), ('bbc', '90b45569a89fb15f'), ('dd5', '19cb472a238245ba'), ('ae4', '5220a7a1b4f8c56c'), ('cbb', 'cba1ac6286025e96'), ('911', 'ef9fa15d98bd5cde'), ('c6f', 'e70f267cc90643df'), ('fb9', '8a6d65d52af0f2d2')]
Fuzzing Attack
Normal     156989
Fuzzing      6972
Name: count, dtype: int64


,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy,Dos Attack,Fuzzing Attack
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354,Normal,Normal
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354,Normal,Normal
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354,Normal,Normal
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354,Normal,Normal
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446,Normal,Normal
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446,Normal,Normal
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446,Normal,Normal
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446,Normal,Normal


# Replaying Label 달기 - 잘못된 코드(이유 : 시간이 서로 맞지 않아 앞에서부터 동일한 ID Data가 나오는 경우를 모두 잡게됨)

In [93]:
# 공백 제거 및 소문자로 변환하여 데이터 포맷 일치
df['Data'] = df['Data'].str.replace(" ", "").str.lower()
df['ID'] = df['ID'].astype(str).str.lower()
df_replaying['Data'] = df_replaying['Data'].str.replace(" ", "").str.lower()
df_replaying['ID'] = df_replaying['ID'].astype(str).str.lower()

# 기본적으로 'Normal'로 초기화
df['Replaying Attack'] = 'Normal'

# df_replaying의 각 행을 df에서 한 번씩만 적용
for _, replay_row in df_replaying.iterrows():
    mask = (df['ID'] == replay_row['ID']) & (df['Data'] == replay_row['Data']) & (df['Replaying Attack'] == 'Normal')
    if mask.any():  # 일치하는 행이 존재할 경우
        df.loc[mask.idxmax(), 'Replaying Attack'] = 'Replaying'  # 첫 번째로 일치하는 행에 'Replaying' 설정

# 결과 확인
print(df['Replaying Attack'].value_counts())
df

Replaying Attack
Normal       163375
Replaying       586
Name: count, dtype: int64


,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy,Dos Attack,Fuzzing Attack,Replaying Attack
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354,Normal,Normal,Replaying
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354,Normal,Normal,Replaying
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354,Normal,Normal,Replaying
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354,Normal,Normal,Replaying
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354,Normal,Normal,Replaying
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446,Normal,Normal,Normal
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446,Normal,Normal,Normal
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446,Normal,Normal,Normal
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446,Normal,Normal,Normal


In [157]:
df.to_csv('final labeling.csv', index = False, encoding = 'utf-8')

# Labeling 끝내기 정상 : 0, Dos : 1, Fuzzing Attack : 2, Replaying Attack : 3

In [98]:
df = pd.read_csv('./final labeling.csv')
df

,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy,Dos Attack,Fuzzing Attack,Replaying Attack
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354,Normal,Normal,Replaying
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354,Normal,Normal,Replaying
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354,Normal,Normal,Replaying
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354,Normal,Normal,Replaying
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354,Normal,Normal,Replaying
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446,Normal,Normal,Normal
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446,Normal,Normal,Normal
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446,Normal,Normal,Normal
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446,Normal,Normal,Normal


## !!!11.06 Replaying Attack 라벨 붙이는 방법이 잘 못돼서 이 부분만 다시 수정

In [184]:
df = pd.read_csv('./final_labeling_with_replay.csv')
df

,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy,Dos Attack,Fuzzing Attack,Replaying Attack
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354,Normal,Normal,Normal
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354,Normal,Normal,Normal
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354,Normal,Normal,Normal
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354,Normal,Normal,Normal
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354,Normal,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446,Normal,Normal,Normal
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446,Normal,Normal,Normal
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446,Normal,Normal,Normal
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446,Normal,Normal,Normal


In [186]:
# 새로운 열 'label'을 조건에 따라 설정
df['label'] = df.apply(
    lambda row: 1 if row['Dos Attack'] == 'DoS' else 
                2 if row['Fuzzing Attack'] == 'Fuzzing' else 
                3 if row['Replaying Attack'] == 'Replay' else 0,
    axis=1
)

# 결과 확인
print(df[['Dos Attack', 'Fuzzing Attack', 'Replaying Attack', 'label']].head())
print(df['label'].value_counts())

df

  Dos Attack Fuzzing Attack Replaying Attack  label
0     Normal         Normal           Normal      0
1     Normal         Normal           Normal      0
2     Normal         Normal           Normal      0
3     Normal         Normal           Normal      0
4     Normal         Normal           Normal      0
label
0    132463
1     24000
2      6972
3       526
Name: count, dtype: int64


,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy,Dos Attack,Fuzzing Attack,Replaying Attack,label
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354,Normal,Normal,Normal,0
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354,Normal,Normal,Normal,0
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354,Normal,Normal,Normal,0
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354,Normal,Normal,Normal,0
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354,Normal,Normal,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446,Normal,Normal,Normal,0
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446,Normal,Normal,Normal,0
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446,Normal,Normal,Normal,0
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446,Normal,Normal,Normal,0


In [188]:
df.to_csv("final_labeling_test.csv", index = False, encoding = 'utf-8') 

# 여기서부터 df = pd.read_csv("final_labeling_test.csv")를 사용한다 

# 훈련 모델 선정 및 훈련 진행

In [115]:
df = pd.read_csv("final_labeling_test.csv")
df

,Time,Source,Destination,Protocol,Length,ID,Data,Same Data,Strange Data,Entropy,Dos Attack,Fuzzing Attack,Replaying Attack,label
0,0.000000,0.0,0.0,CAN,32,168,8000080012504000,0,0,3.339354,Normal,Normal,Replaying,3
1,0.001220,0.0,0.0,CAN,32,588,00c6000040001000,0,0,3.339354,Normal,Normal,Replaying,3
2,0.002423,0.0,0.0,CAN,32,589,000000ff9c000000,0,0,3.339354,Normal,Normal,Replaying,3
3,0.003685,0.0,0.0,CAN,32,178,0000000000000000,0,0,3.339354,Normal,Normal,Replaying,3
4,0.011932,0.0,0.0,CAN,32,101,9000000000000100,0,0,3.339354,Normal,Normal,Replaying,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163956,641.001177,0.0,0.0,CAN,32,147,0000000000000000,0,0,3.741446,Normal,Normal,Normal,0
163957,641.010075,0.0,0.0,CAN,32,112,c000001f04800000,0,0,3.741446,Normal,Normal,Normal,0
163958,641.012440,0.0,0.0,CAN,32,188,98a0000000000000,0,0,3.741446,Normal,Normal,Normal,0
163959,641.014382,0.0,0.0,CAN,32,141,3000000000000000,0,0,3.741446,Normal,Normal,Normal,0


In [127]:
label = df['label']
label

0         3
1         3
2         3
3         3
4         3
         ..
163956    0
163957    0
163958    0
163959    0
163960    0
Name: label, Length: 163961, dtype: int64

In [123]:
data = df.drop(['Time','Source','Destination','Protocol','Length','ID','Data'
                , 'Dos Attack','Fuzzing Attack','Replaying Attack','label'], axis = 1)

In [125]:
data

,Same Data,Strange Data,Entropy
0,0,0,3.339354
1,0,0,3.339354
2,0,0,3.339354
3,0,0,3.339354
4,0,0,3.339354
...,...,...,...
163956,0,0,3.741446
163957,0,0,3.741446
163958,0,0,3.741446
163959,0,0,3.741446


## KNN 알고리즘 적용

In [129]:
from sklearn.model_selection import train_test_split

tr_X, te_X, tr_y, te_y = train_test_split(data, label, test_size = 0.2, random_state = 200)

tr_X.shape, te_X.shape, tr_y.shape, te_y.shape

((131168, 3), (32793, 3), (131168,), (32793,))

In [131]:
# KNN 알고리즘 사용
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 3)

model.fit(tr_X, tr_y)

KNeighborsClassifier(n_neighbors=3)

In [133]:
model.score(te_X, te_y)

0.9542890250968195

## 정규화 후 KNN 알고리즘

In [137]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

sc.fit(data)

norm_data = sc.transform(data)

norm_df = pd.DataFrame(norm_data, columns=data.columns)

norm_df.describe()

tr_X, te_X, tr_y, te_y = train_test_split(norm_df, label, test_size=0.2, random_state=200)

model = KNeighborsClassifier(n_neighbors=3)

model.fit(tr_X, tr_y)

model.score(te_X, te_y)

0.9534961729637422

## RandomForest 적용

In [142]:
from sklearn.ensemble import RandomForestClassifier
 
clf = RandomForestClassifier(max_depth=150, random_state=100)

clf.fit(tr_X, tr_y)

clf.score(te_X, te_y)

0.9536181502149849

In [173]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('./1029_attack.csv')  # 원래 CSV 파일 경로

# ID가 연속으로 같은 값인 경우 찾기
duplicate_ids = df[df['ID'] == df['ID'].shift(1)]

# 중복된 ID만 따로 CSV 파일로 저장
duplicate_ids.to_csv('duplicate_ids.csv', index=False)

print("중복된 ID 행을 'duplicate_ids.csv' 파일로 저장했습니다.")


중복된 ID 행을 'duplicate_ids.csv' 파일로 저장했습니다.


In [182]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('./final labeling.csv')  # 파일 이름에 맞게 수정하세요

# 새로운 열 추가 - 기본값을 'Normal'로 설정
df['Replaying Attack'] = 'Normal'

# 주어진 행 범위
ranges = [(1640, 3239), (10118, 13299), (41052, 42905), (48259, 50235),(68195,103801), (144926, 147086)]

# 각 범위 내에서 연속된 중복 ID에 대해 'Replay'로 레이블링
for start, end in ranges:
    # 범위에 해당하는 데이터 선택
    subset = df.iloc[start:end + 1]  # Pandas는 끝값을 포함하지 않으므로 +1 필요
    # 연속된 중복 ID 찾기
    replay_indices = subset[subset['ID'] == subset['ID'].shift(1)].index
    # 해당 인덱스에 'Replay' 표시
    df.loc[replay_indices, 'Replaying Attack'] = 'Replay'

# 결과 저장
df.to_csv('final_labeling_with_replay.csv', index=False)

print("Replaying Attack 열을 추가한 'final_labeling_with_replay.csv' 파일이 생성되었습니다.")


Replaying Attack 열을 추가한 'final_labeling_with_replay.csv' 파일이 생성되었습니다.
